# Modelo

In [1]:
nomemodelo = 'Random Forest'

## Caminhos

In [2]:
import os

#Folder Inicial
path = os.getcwd()

#Subpastas
pathin = path + '\\Entrada\\'
pathfixo = path + '\\Fixo\\'
pathout = path + '\\Saida\\'
pathparcial = path + '\\Parcial\\'
pathaux = path + '\\Auxiliar\\'

## Pacotes

In [3]:
import dill
import pickle
import pandas as pd
pd.set_option('max_colwidth', 3000)

import numpy as np

from time import gmtime, strftime

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import lightgbm as lgb

from sklearn.externals import joblib

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.utils import parallel_backend

In [4]:
import sklearn
print(sklearn.__version__)

0.20.3


## Lendo a Base

In [5]:
file = 'Treino'
treino = pd.read_pickle(pathparcial + 'Arquivo0 ' + file + '.pkl')

file = 'Teste'
teste = pd.read_pickle(pathparcial + 'Arquivo0 ' + file + '.pkl')

print(treino.shape)
print(teste.shape)

(750, 58)
(250, 58)


In [6]:
#Excluindo variaveis que nao serao usadas no modelo
y = 'default'
id = 'id'

Xtr = treino.drop([y, id], axis = 1)
colunas = list(Xtr)

with open(pathaux + 'Variaveis Modelo ' + nomemodelo + '.pickle', 'wb') as f:
    dill.dump((colunas, y), f)

Xte = teste[colunas]

print(Xtr.shape)
print(Xte.shape)

(750, 56)
(250, 56)


## Modelos

In [7]:
nomemodelo = 'Random Forest'

seed = 123

parameters = {
    'n_estimators': [100, 300, 500, 700],
    'min_samples_leaf': [5, 10],
    'class_weight': ['balanced', None],
    'random_state': [seed]
}
        
estimator = RandomForestClassifier()

cv = RepeatedStratifiedKFold(n_splits = 3, n_repeats = 1, random_state = seed)

gs = GridSearchCV(estimator = estimator, param_grid = parameters, cv = cv, scoring = 'f1_macro', 
                  n_jobs = 3, verbose = 1, refit = True)

In [8]:
#Rodando GridSearch
print(strftime('%Y-%m-%d %H:%M:%S', gmtime()))

with parallel_backend('multiprocessing'):
    gs.fit(Xtr, treino[y])

joblib.dump(gs, pathaux + 'Modelo ' + nomemodelo + '.pkl')

print(strftime('%Y-%m-%d %H:%M:%S', gmtime()))

2019-05-12 01:41:02
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=3)]: Using backend MultiprocessingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  48 out of  48 | elapsed:   15.1s finished


2019-05-12 01:41:23


In [9]:
gs.best_score_

0.7185572030229127

In [10]:
gs.best_params_

{'class_weight': 'balanced',
 'min_samples_leaf': 5,
 'n_estimators': 300,
 'random_state': 123}

In [11]:
cvresults = pd.DataFrame(gs.cv_results_)[[
 'mean_test_score',
 'mean_train_score',
 'param_n_estimators',
 'param_min_samples_leaf',
 'param_class_weight',
 'std_test_score',
 'std_train_score']]
ha = list(cvresults)
ha = [w.replace('param_', '') for w in ha]
cvresults.columns = ha
cvresults.to_excel(pathout + 'Modelo ' + nomemodelo + ' Resultados GridSearch.xlsx', encoding = 'latin1', index = False)

C:\Users\laran\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\laran\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\laran\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\laran\Anaconda3\lib\site-packages\sklearn\util

In [12]:
cvresults

,mean_test_score,mean_train_score,n_estimators,min_samples_leaf,class_weight,std_test_score,std_train_score
0,0.702489,0.847840,100,5,balanced,0.022517,0.013150
1,0.718557,0.857084,300,5,balanced,0.029112,0.004529
2,0.705033,0.855882,500,5,balanced,0.024714,0.001835
3,0.705568,0.853532,700,5,balanced,0.021391,0.003386
4,0.698039,0.764300,100,10,balanced,0.036531,0.005392
5,0.684505,0.771813,300,10,balanced,0.028982,0.005746
6,0.683053,0.771389,500,10,balanced,0.027615,0.010733
7,0.682456,0.773104,700,10,balanced,0.023204,0.008783
8,0.602121,0.808227,100,5,None,0.035339,0.013786
9,0.598734,0.806410,300,5,None,0.036097,0.014060


## Variaveis Importantes

In [13]:
def Feature_Importance(mod, nomemodelo, X, y):
    many = ['Linear SVM', 'Logistica', 'Naive Bayes']

    k = mod.best_estimator_
    featurenames = list(X)
    
    if nomemodelo in many:
        #Logistica, SVM, Naive Bayes
        Features = pd.DataFrame(k.coef_.tolist())
        Features.columns = featurenames
        if len(y.unique()) > 2:
            Features.index = sorted(y.unique())
    else:
        #Arvores
        Features = pd.DataFrame({'features': featurenames, 'value': k.feature_importances_.tolist()})
        
    Features.to_excel(pathout + 'Features ' + nomemodelo + '.xlsx', encoding = 'latin1', index = True)
    
Feature_Importance(mod = gs, nomemodelo = nomemodelo, X = Xtr, y = treino[y])

## Preditos

In [14]:
predtr = gs.best_estimator_.predict(Xtr)
predte = gs.best_estimator_.predict(Xte)

In [15]:
name = 'Treino'
fim = treino[[id, y]]
fim['pred'] = predtr

fim['Acertou'] = np.where(fim[y] == fim['pred'], 1, 0)
fim.to_excel(pathout + 'Modelo ' + nomemodelo + ' Pred ' + name + '.xlsx', encoding = 'latin1', index = False)

del fim

name = 'Teste'
fim = teste[[id, y]]
fim['pred'] = predte

fim['Acertou'] = np.where(fim[y] == fim['pred'], 1, 0)
fim.to_excel(pathout + 'Modelo ' + nomemodelo + ' Pred ' + name + '.xlsx', encoding = 'latin1', index = False)

C:\Users\laran\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laran\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\laran\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 